<a href="https://colab.research.google.com/github/Ayushimishra02/bulb.css/blob/main/emodb_datset_svc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d piyushagni5/berlin-database-of-emotional-speech-emodb



In [ ]:
!unzip -o berlin-database-of-emotional-speech-emodb.zip -d emodb_data

In [ ]:
import os #for making path for file folder in this case the speech files
import librosa #for conversion of speech files into melspectograms
import librosa.display #to display as melspectograms to understand it
import matplotlib.pyplot as plt
data_dir="emodb_data"
emotion_map={
    'W':'Angry','L':'Bored','E':'Disgust',
    'A':'Fearful','F':'Happy','T':'Sad','N':'Neutral'
}

In [ ]:
def extract_features(file_path): #we extract features from the file path so we can use it for every audio fiiles
  y,sr=librosa.load(file_path,sr=160000) #y is the number that represent how loud is the sound at each momemt and 16000 is the no.of samples persecond which is voice resolution
  spect=librosa.feature.melspectrogram(y=y,sr=sr,n_mels=128) #mel_spects 128 is how detailed the image would be would be and melspectogram means making a clear and good quality picture of graph of speech noices
  #mel means the pitch recorded and heard like a human not robot
  #converting the picture from raw intensity to decibel scale means making the high pitch places brighter and less noise places darker
  spect_db=librosa.power_to_db(spect,ref=np.max)
  return spect_db

In [ ]:
#librosa is python library for music and audio analytics

In [ ]:
import os #helps in providing files or folders path in ur computer
import numpy as np
X=[] #x has feature in vector fomr
Y=[] #y has emotion label of angry or happy etc.
for root, _, files in os.walk(data_dir): #os.walk helps in iterating through each folder and its subfolder and its next part inside sub folder so in one line you get all ur audio files and u do not need to manually import all of them here
    for file in files:#_use the files which we do not use files here thorway files
      if file.endswith('.wav'): #.wav is the audio classification of all the datas inside the datset it makes sure that no other format datas get here
        file_path=os.path.join(root,file) #os.path.join makes sure that every .wav file path is used here it give the exact location of every .wav file in them and also helps in running it in every possible os like windows,linux,macos
        emotion_code = file[5] #5 means use the 6th character in the file name that is its actual emotion calling like "03010L01"
        if emotion_code in emotion_map: #and whatever that 5th code is it checks if it is in the emotion map
            emotion = emotion_map[emotion_code] #if its there add i to emotion tab what its actual value is not key
            #print(f"File: {file}, Emotion: {emotion}") #then print it
            spect_db = extract_features(file_path)#it helps in extracting all features of graphs in file_path
            #librosa.display.specshow(spect_db, sr=160000, x_axis='time', y_axis='mel')#librosa.display.specshow makes the spectograms and 16000 iis no.of samples taken at a time
            #plt.colorbar(format='%+2.0f dB')#colobar helps in providing a colorbar at the side of the graph showing darkis equal to loud sound and soft colors is normal low sounds
            #plt.title(f'Mel-spectrogram for {emotion}')
            #plt.show() #plt is like plotting a graph based on given instructions
            feature_vector=np.mean(spect_db,axis=1) #we are making a 2d file into 1d to flatten it so axis of time is only here becuase x was time axis
            X.append(feature_vector)
            Y.append(emotion_map[emotion_code])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report
X=np.array(X)
Y=np.array(Y)
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.22) #random state 42 ensures you get the same split everytime u run it
clf=SVC(kernel='linear',C=1) #linear uses st. line to sperate class and c=1 balances accuracy and simplicity
clf.fit(X_train,y_train) #model learns from pattern x_train to predict on y_train
y_pred=clf.predict(X_test)
print("Accuracy",accuracy_score(y_test,y_pred))
print("Classification_report",classification_report(y_test,y_pred))

In [ ]:
#in output it is precision meaning out of so many sounds how precise u were to classify if the sound is happy ,sad
#recall is how much of the actual particular tracks(happy,sad..), model was able to vatch coorectly
#f1 is basically calculating mean value for the precision and recall

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
import numpy as np
import librosa
file_name=list(uploaded.keys())[0] #i want my predicting model to give svm the name of the file we are uploading without the name svm wont know to work with wht
features=extract_features(file_name)
feature_vector=np.mean(features,axis=1)
feature_vector=feature_vector.reshape(1,-1)
predicted_emotion_code=clf.predict(feature_vector)[0]
reverse_map={v:k for k,v in emotion_map.items()}
predicted_emotion=reverse_map.get(predicted_emotion_code,"unknown")
print(f"predicted emotion:{predicted_emotion_code},predicted emotion code:{predicted_emotion}")

